In [1]:
import nilmtk
import pandas as pd

In [2]:
from nilmtk import DataSet

In [3]:
iawe = DataSet('iawe.h5')

In [4]:
elec = iawe.buildings[1].elec
elec

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='iAWE', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='iAWE', site_meter, appliances=[])
  ElecMeter(instance=3, building=1, dataset='iAWE', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=4, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=1)])
  ElecMeter(instance=5, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=2)])
  ElecMeter(instance=6, building=1, dataset='iAWE', appliances=[Appliance(type='washing machine', instance=1)])
  ElecMeter(instance=7, building=1, dataset='iAWE', appliances=[Appliance(type='computer', instance=1)])
  ElecMeter(instance=8, building=1, dataset='iAWE', appliances=[Appliance(type='clothes iron', instance=1)])
  ElecMeter(instance=9, building=1, dataset='iAWE', appliances=[Appliance(type='unknown', instance=1)])
  ElecMeter(instance=10, building=1, dataset='iAWE', appliances=[A

In [5]:
fridge = elec['fridge']
fridge.available_columns()

[('power', 'reactive'),
 ('power', 'apparent'),
 ('current', None),
 ('power factor', None),
 ('frequency', None),
 ('voltage', None),
 ('power', 'active')]

In [7]:
df = next(fridge.load())
df.head()

physical_quantity            power          current  frequency     voltage  \
type                      reactive apparent                                  
2013-06-07 05:30:00+05:30    2.483    2.486   0.011  50.070000  235.070007   
2013-06-07 05:30:01+05:30    2.547    2.555   0.011  50.080002  235.020004   
2013-06-07 05:30:02+05:30    2.480    2.485   0.011  50.080002  234.979996   
2013-06-07 05:30:03+05:30    2.444    2.449   0.010  50.060001  235.000000   
2013-06-07 05:30:04+05:30    2.510    2.519   0.011  50.060001  234.949997   

physical_quantity          power  
type                      active  
2013-06-07 05:30:00+05:30  0.111  
2013-06-07 05:30:01+05:30  0.200  
2013-06-07 05:30:02+05:30  0.152  
2013-06-07 05:30:03+05:30  0.159  
2013-06-07 05:30:04+05:30  0.215

In [8]:
mi = df.columns

In [9]:
mi

MultiIndex([(    'power', 'reactive'),
            (    'power', 'apparent'),
            (  'current',         ''),
            ('frequency',         ''),
            (  'voltage',         ''),
            (    'power',   'active')],
           names=['physical_quantity', 'type'])

In [21]:
tb = pd.DataFrame(columns = df.columns)

In [22]:
new_df = pd.read_csv('ac_seconds.csv',names =df.columns)

In [23]:
new_df.head()

power          current frequency voltage  power
                          reactive apparent                           active
2020-08-12 00:00:00+05:30  3000.00   3000.0    9.94     49.98  230.07   -3.0
2020-08-12 00:00:01+05:30  2500.00   2500.0    9.93     49.98  230.15   -5.0
2020-08-12 00:00:02+05:30  2799.97   2800.0    9.92     49.98  230.15  -14.0
2020-08-12 00:00:03+05:30  2599.97   2600.0    9.89     49.98  230.15  -13.0
2020-08-12 00:00:04+05:30  2374.92   2375.0    9.88     49.98  230.14  -19.0

In [24]:
tb[df.columns]= new_df[new_df.columns]
tb.head()

physical_quantity            power          current frequency voltage  power
type                      reactive apparent                           active
2020-08-12 00:00:00+05:30  3000.00   3000.0    9.94     49.98  230.07   -3.0
2020-08-12 00:00:01+05:30  2500.00   2500.0    9.93     49.98  230.15   -5.0
2020-08-12 00:00:02+05:30  2799.97   2800.0    9.92     49.98  230.15  -14.0
2020-08-12 00:00:03+05:30  2599.97   2600.0    9.89     49.98  230.15  -13.0
2020-08-12 00:00:04+05:30  2374.92   2375.0    9.88     49.98  230.14  -19.0

tb.to_hdf('CAXE.h5', key='CAXE', mode='w')

In [25]:
import numpy as np
from os.path import join
from nilmtk.datastore import Key
from nilmtk.measurement import LEVEL_NAMES
from nilmtk.utils import check_directory_exists, get_datastore, get_module_directory
from nilm_metadata import convert_yaml_to_hdf5
from copy import deepcopy

In [37]:
TIMESTAMP_COLUMN_NAME = "timestamp"
TIMEZONE = "Asia/Kolkata"
START_DATETIME, END_DATETIME = '2020-08-12', '2020-08-12'
FREQ = "S"

output_filename = 'caxe25'
"""
Parameters
----------
iawe_path : str
    The root path of the iawe dataset.
output_filename : str
    The destination filename (including path and suffix).
"""

# Open data store
store = get_datastore(output_filename, format='HDF', mode='w')

key = Key(building=1, meter=1)
print('Loading ', 1)

df =tb
df = df.dropna()
df = df.astype(np.float32)
df = df.sort_index()

assert df.isnull().sum().sum() == 0
store.put(str(key), df)
store.close()

convert_yaml_to_hdf5('./metadata', output_filename)

print("Done converting iAWE to HDF5!")

Loading  1
Done converting YAML metadata to HDF5!
Done converting iAWE to HDF5!


In [168]:
from nilmtk import DataSet
from nilmtk.utils import print_dict

caxe = DataSet('./caxe9')

In [18]:
store.close

<function nilmtk.datastore.hdfdatastore.DocInherit.get_with_inst.<locals>.f>

In [169]:
caxe

In [171]:
caxe.buildings

OrderedDict([(1, Building(instance=1, dataset='iAWE'))])

In [28]:
caxe.metadata

NameError: name 'caxe' is not defined